In [4]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']




In [5]:

"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
calendar_token_loc = 'creds_google/calendar-token.pickle'
credentials_loc = 'creds_google/credentials.json'

if os.path.exists(calendar_token_loc):
    with open(calendar_token_loc, 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            credentials_loc, SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(calendar_token_loc, 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)
# Call the Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()

events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])



Getting the upcoming 10 events
2021-01-22T22:00:00+03:00 IBM Data Science
2021-01-23T17:00:00+03:00 Empty
2021-01-23T22:00:00+03:00 IBM Data Science
2021-01-24T13:00:00+03:00 Work
2021-01-24T17:00:00+03:00 Empty
2021-01-24T20:00:00+03:00 Resources : Weekly Checklist DDL
2021-01-24T21:00:00+03:00 EB SHA3'AF MEETING
2021-01-24T22:00:00+03:00 IBM Data Science
2021-01-25T17:00:00+03:00 Empty
2021-01-25T22:00:00+03:00 IBM Data Science


In [26]:
event = {'summary': 'Control', 'location': 'Virtual', 'start': {'dateTime': '2021-01-22T17:00:00+03:00', 'timeZone': 'GMT+03:00'}, 'end': {'dateTime': '2021-01-22T17:30:00+03:00', 'timeZone': 'GMT+03:00'}, 'recurrence': ['RRULE:FREQ=DAILY;UNTIL=20210125T235959Z']}

code = 'ftt7hrhctc3rt8vlmjmfaavl1o'

In [27]:
update_event(event, code)

The event was updated at 2021-01-22T15:36:01.409Z
summary : Control


In [43]:
{'dateTime': '2021-01-22T17:00:00.000', 'timeZone': 'GMT+3:00'} == {'dateTime': '2021-01-22T17:00:00.000', 'timeZone': 'GMT+3:00'}

True

In [24]:
def update_event(proposed_event, event_code):
    
    changed_fields = []
    current_event = service.events().get(calendarId='primary', eventId=event_code).execute()
    
    # Check to see if there is any change
    for field in proposed_event:
        if proposed_event[field] != current_event[field]:
            booop = current_event[field]
            beeep = proposed_event[field]
            current_event[field] = proposed_event[field]
            changed_fields.append(f'{field} : {current_event[field]}')
    
    
    # Update the event        
    if len(changed_fields) > 0:
        updated_event = service.events().update(calendarId='primary', eventId=event_code, body=current_event).execute()
        print(f"The event was updated at {updated_event['updated']}")
        for field in changed_fields: print(field)
    else:
        print(f"No change to {current_event['summary']} was necessary")
                                  
    

{'dateTime': '2021-01-22T17:30:00.000', 'timeZone': 'GMT+3:00'} {'dateTime': '2021-01-22T17:30:00+03:00', 'timeZone': 'GMT+03:00'}


In [28]:
print(_.keys())
_['start']

dict_keys(['attachments', 'creator', 'organizer', 'summary', 'id', 'hangoutLink', 'attendees', 'start', 'htmlLink', 'recurrence', 'source', 'etag', 'location', 'recurringEventId', 'gadget', 'status', 'updated', 'description', 'iCalUID', 'extendedProperties', 'endTimeUnspecified', 'sequence', 'visibility', 'guestsCanModify', 'end', 'attendeesOmitted', 'kind', 'locked', 'anyoneCanAddSelf', 'colorId', 'created', 'reminders', 'guestsCanSeeOtherGuests', 'originalStartTime', 'guestsCanInviteOthers', 'transparency', 'privateCopy', 'conferenceData'])


{'date': 'A String', 'timeZone': 'A String', 'dateTime': 'A String'}

In [6]:
event = {
  'summary': 'Appointment',
  'location': 'Somewhere',
  'start': {
    'dateTime': '2021-01-20T10:00:00.000',
    'timeZone': 'GMT+3:00'
  },
  'end': {
    'dateTime': '2021-01-20T10:25:00.000',
      'timeZone': 'GMT+3:00'
  },
  'recurrence': [
    'RRULE:FREQ=DAILY;UNTIL=20210122T235959Z',
  ],
}
recurring_event = service.events().insert(calendarId='primary', body=event).execute()

print( recurring_event['id'])

hh3tv9od5rbpd0jue2dib69ssc
